In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, scrolledtext
import pywhatkit 
import pyttsx3
import os
import cv2
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter
import numpy as np
import matplotlib.pyplot as plt
import geocoder
from twilio.rest import Client
from googlesearch import search
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

def open_calendar():
    print("Opening calendar...")
    os.system("start outlookcal:")

def open_whatsApp():
    print("opening WhatsApp for you" )
    os.system("start WhatsApp")
    pywhatkit.sendwhatmsg_instantly("+91","hello this is a message using python",20,31) #write phone number

# Functionality for Laptop Volume Control
def set_volume():
        try:
            volume_level = simpledialog.askfloat("Volume Control", "Enter volume level (in dB):", minvalue=-96.0, maxvalue=0.0)
            if volume_level is not None:
                devices = AudioUtilities.GetSpeakers()
                interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
                volume = cast(interface, POINTER(IAudioEndpointVolume))
                volume.SetMasterVolumeLevel(volume_level, None)
                messagebox.showinfo("Volume Control", f"Volume set to {volume_level} dB")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to set volume: {e}")


# Function to perform Google search
def google_search():
    query = simpledialog.askstring("Google Search", "Enter search query:")
    try:
        results = search(query, num_results=5)
        results_text = "\n".join([f"{i + 1}. {result}" for i, result in enumerate(results)])
        messagebox.showinfo("Google Search Results", results_text)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to perform search: {str(e)}")


# Functionality for Text to Speech
def text_to_speech():
    text = simpledialog.askstring("Text to Speech", "Enter text to convert to speech:")
    if text:
        engine = pyttsx3.init()
        engine.say(text)
        engine.runAndWait()
        messagebox.showinfo("Text to Speech", f"Speaking: {text}")

# Functionality for Capture Image and Apply Sunglasses Filter
def capture_and_apply_filter():
    capture_image('captured_image.jpg')
    apply_realistic_sunglasses_filter('captured_image.jpg')

def capture_image(filename='captured_image.jpg'):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Could not access the webcam.")
        return
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(filename, frame)
        messagebox.showinfo("Image Capture", f"Image saved as {filename}")
    else:
        messagebox.showerror("Error", "Could not capture an image.")
    cap.release()
    cv2.destroyAllWindows()

def apply_realistic_sunglasses_filter(image_path):
    image = Image.open(image_path).convert('RGBA')
    width, height = image.size
    overlay = Image.new('RGBA', image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)
    glasses_width = width // 3
    glasses_height = height // 10
    bridge_width = width // 10
    bridge_height = glasses_height // 2
    left_eye_center = (width // 3, height // 3)
    right_eye_center = (2 * width // 3, height // 3)
    bridge_center = (width // 2, height // 3 + glasses_height // 2)
    left_bbox = [left_eye_center[0] - glasses_width // 2, left_eye_center[1] - glasses_height // 2,
                 left_eye_center[0] + glasses_width // 2, left_eye_center[1] + glasses_height // 2]
    draw.ellipse(left_bbox, fill=(0, 0, 0, 128))
    right_bbox = [right_eye_center[0] - glasses_width // 2, right_eye_center[1] - glasses_height // 2,
                  right_eye_center[0] + glasses_width // 2, right_eye_center[1] + glasses_height // 2]
    draw.ellipse(right_bbox, fill=(0, 0, 0, 128))
    bridge_bbox = [bridge_center[0] - bridge_width // 2, bridge_center[1] - bridge_height // 2,
                   bridge_center[0] + bridge_width // 2, bridge_center[1] + bridge_height // 2]
    draw.rectangle(bridge_bbox, fill=(0, 0, 0, 128))
    image_with_sunglasses = Image.alpha_composite(image, overlay)
    image_with_sunglasses = image_with_sunglasses.convert('RGB')
    image_with_sunglasses.save('realistic_sunglasses_filter.jpg')
    image_with_sunglasses.show()

# Functionality for Send SMS
def send_sms():
    to_phone_number = simpledialog.askstring("Send SMS", "Enter recipient phone number:")
    message_body = simpledialog.askstring("Send SMS", "Enter message body:")
    if to_phone_number and message_body:
        try:
            account_sid = 'your_account_sid' #write your account sid
            auth_token = 'your_auth_token'#write your account auth token
            client = Client(account_sid, auth_token)
            from_phone_number = 'Twilio_phone_number' ##write your twilio phone number
            message = client.messages.create(
                body=message_body,
                from_=from_phone_number,
                to=to_phone_number
            )
            messagebox.showinfo("Send SMS", f"Message sent with SID: {message.sid}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

# Functionality for Location and Geo Coordinates
def show_location():
    g = geocoder.ip('me')
    location_info = f"Latitude: {g.lat}\nLongitude: {g.lng}\nAddress: {g.address}"
    messagebox.showinfo("Location", location_info)

# Functionality for Image Filters
def apply_filters():
    image_path = filedialog.askopenfilename(title="Select Image File", filetypes=[("Image Files", "*.jpg;*.png")])
    if image_path:
        image = Image.open(image_path)
        blurred_image = image.filter(ImageFilter.BLUR)
        sharpened_image = image.filter(ImageFilter.SHARPEN)
        edge_enhanced_image = image.filter(ImageFilter.EDGE_ENHANCE)
        grayscale_image = image.convert("L")
        enhancer = ImageEnhance.Brightness(image)
        brightened_image = enhancer.enhance(1.5)

        plt.figure(figsize=(12, 8))
        plt.subplot(2, 3, 1)
        plt.imshow(image)
        plt.title("Original")
        plt.axis('off')
        plt.subplot(2, 3, 2)
        plt.imshow(blurred_image)
        plt.title("Blurred")
        plt.axis('off')
        plt.subplot(2, 3, 3)
        plt.imshow(sharpened_image)
        plt.title("Sharpened")
        plt.axis('off')
        plt.subplot(2, 3, 4)
        plt.imshow(edge_enhanced_image)
        plt.title("Edge Enhanced")
        plt.axis('off')
        plt.subplot(2, 3, 5)
        plt.imshow(grayscale_image, cmap='gray')
        plt.title("Grayscale")
        plt.axis('off')
        plt.subplot(2, 3, 6)
        plt.imshow(brightened_image)
        plt.title("Brightened")
        plt.axis('off')
        plt.show()

# Functionality for Create a Custom Image
def create_custom_image():
    color_image = np.zeros((100, 100, 3), dtype=np.uint8)
    color_image[40:60, 40:60, 0] = 255
    plt.imshow(color_image)
    plt.title("Custom Image")
    plt.axis('off')
    plt.show()

# Functionality for Crop Image
def crop_image():
    capture_image('captured_image.jpg')
    face_bbox = detect_face('captured_image.jpg')
    if face_bbox:
        image = Image.open('captured_image.jpg')
        cropped_face = image.crop((face_bbox[0], face_bbox[1], face_bbox[0] + face_bbox[2], face_bbox[1] + face_bbox[3]))
        cropped_face.save('cropped_face.jpg')
        overlay_face_on_image('captured_image.jpg', 'cropped_face.jpg', face_bbox)

def detect_face(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        messagebox.showerror("Error", "No face detected.")
        return None
    x, y, w, h = faces[0]
    return (x, y, w, h)

def overlay_face_on_image(original_image_path, face_image_path, face_bbox):
    original_image = Image.open(original_image_path)
    face_image = Image.open(face_image_path)
    face_image = face_image.resize((face_bbox[2], face_bbox[3]))
    overlay = Image.new('RGBA', original_image.size, (0, 0, 0, 0))
    overlay.paste(face_image, (face_bbox[0], face_bbox[1]))
    combined_image = Image.alpha_composite(original_image.convert('RGBA'), overlay)
    combined_image = combined_image.convert('RGB')
    combined_image.save('overlayed_face_image.jpg')
    combined_image.show()

# Functionality for Make a Phone Call
def make_phone_call():
    to_phone_number = simpledialog.askstring("Make Phone Call", "Enter recipient phone number:")
    if to_phone_number:
        try:
            account_sid = 'your_account_sid' #write your account sid
            auth_token = 'your_auth_token'#write your account auth token
            client = Client(account_sid, auth_token)
            from_phone_number = 'Twilio_phone_number' ##write your twilio phone number
            twiml_url = 'http://demo.twilio.com/docs/voice.xml'
            call = client.calls.create(
                to=to_phone_number,
                from_=from_phone_number,
                url=twiml_url
            )
            messagebox.showinfo("Phone Call", f"Call initiated with SID: {call.sid}")
        except Exception as e:
            messagebox.showerror("Error", str(e))


# Tkinter GUI setup
root = tk.Tk()
root.title("Portfolio Automation")

# Set a sky blue background
root.configure(bg='sky blue')

# Add name and welcome message
header_frame = tk.Frame(root, bg='sky blue')
header_frame.pack(pady=80)

tk.Label(header_frame, text="Anjali Sharma", font=('Mongolian baiti', 26, 'bold'), bg='sky blue').pack()
tk.Label(header_frame, text="Welcome to my project!", font=('Forte', 16), bg='sky blue').pack()

# Frame for buttons
frame = tk.Frame(root, bg='sky blue')
frame.pack(pady=20)

# Button styling
button_style = {
    'width': 30,
    'height': 2,
    'bg': '#003366',  # Dark blue color
    'fg': 'white',    # White text
    'font': ('Arial', 12, 'bold')
}

# Map button texts to their respective functions
button_commands = {
    "Set Volume": set_volume,
    "Google Search": google_search,
    "Text-to-Speech": text_to_speech,
    "Capture Image": capture_and_apply_filter,
    "Send SMS": send_sms,
    "Get Location": show_location,
    "Apply Filters": apply_filters,
    "Create Custom Image": create_custom_image,
    "Crop Image": crop_image,
    "Make Phone Call": make_phone_call,
    "WhatsApp messsage": open_whatsApp,
    "Open Calendar": open_calendar,
    # "Assistance": assistance
}


# Create buttons dynamically
for i, (text, command) in enumerate(button_commands.items()):
    tk.Button(frame, text=text, command=command, **button_style).grid(row=i//2, column=i%2, padx=5, pady=5)

# Run the application
root.mainloop()
